In [24]:
%matplotlib inline
import sys
import lxml
import requests
import time
from tqdm import tqdm
from bs4 import BeautifulSoup as bs

import numpy as np
import pandas as pd

import matplotlib
from matplotlib import pyplot as plt
import re

import psycopg2
from psycopg2 import OperationalError
from io import BytesIO
from PIL import Image
from IPython.display import display

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.action_chains import ActionChains

In [4]:
login heritageauctions :
    
trololo@guerrillamail.org
username : trololo
trololo123AZE

SyntaxError: invalid syntax (2722948313.py, line 1)

In [5]:
import functions

In [101]:
# launch driver with auth URL

url = "https://jewelry.ha.com/c/login.zx?source=nav"
driver_ = functions.driver_init(login_url = url)

An error occurred: Message: ; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception



In [8]:
# Navigate to designer marks URL 
designermarks_href="https://jewelry.HA.com/ref/designer-marks.zx?ic=Tab-Resources-DesignerMarks-121516-interior"
driver_.get(designermarks_href)
driver_.implicitly_wait(2)

In [10]:
# Get HTML Code from current page. Allows us to get all designer names
html_code = driver_.page_source
driver_.implicitly_wait(2)
soup = bs(html_code, 'lxml')

# Part 1 : Get Designer Names, and reference to their page
This will allow us to access their page and products later

Cocher aussi la case "not sold" si elle est disponible, permet d'avoir tous les résultats

Récupérer les url vers la page du produit, en fonction du auction ID, du lot ID, de l'url vers le produit

In [ ]:
# Bug below, parfois prend l'url d'un produit, mettre une condition sur le findall?

In [43]:

# Creates a list of dictionnaries. Each dict contains name, url and image_url (later replaced by image binary data).
designers=[]
for i in soup.find_all(class_ = 'mark-title'):
    #print(i.contents)
    designer = {}
    designer["name"] = i.find('a').text
    designer["page_url"] = i.find('a')['href']
    designers.append(designer)


In [54]:
designers[4]

{'name': 'Adler',
 'page_url': 'https://jewelry.HA.com/c/search.zx?header=Estate+Jewelry&search=archives&txtSearch=adler&ic22=JewelryMM-DesignerName-061120'}

In [52]:
# Backup Data to disk
# functions.pickle_data('designers.pkl', 'write', designers) #Write data
# my_data = functions.pickle_data('designers.pkl', 'read') #Read data
functions.pickle_data('designers.pkl', 'write', designers) #Write data

# Part 2 : Navigate Designer pages & extract data
Click bottom 200 results, allows us to show all results

Cocher aussi la case "not sold" si elle est disponible, permet d'avoir tous les résultats

Récupérer les url vers la page du produit, en fonction du auction ID, du lot ID, de l'url vers le produit

In [61]:
#test to see if data is correctly saved
driver_.get(designers[7].get('page_url'))

In [64]:
#test to get url of a designer
designers[7].get('page_url')

'https://jewelry.HA.com/c/search.zx?header=Estate+Jewelry&search=archives&txtSearch=aletto+brothers&ic22=JewelryMM-DesignerName-061120'

In [ ]:
# click on "not sold" to get all items
to do

In [65]:
# Get list of all products 
html_code = driver_.page_source
driver_.implicitly_wait(2)
soup = bs(html_code, 'lxml')

In [93]:
# For each designer page 
sales=[]
for sale in soup.find_all(class_ = 'item-block list'):
    products={}
    #print(sale.contents)
    sale_url=sale.find('a')['href']
    
    #Extract Lot Number and Auction number
    sale_text= sale.find(class_ = 'item-info').text
    auction_number = re.search(r'Auction .*?(\d+)', sale_text).group(1)
    lot_number = re.search(r'Lot .*?(\d+)', sale_text).group(1)
    products["sale_url"] = sale_url
    products["auction_number"] = auction_number
    products["lot_number"] = lot_number
    
    sales.append(products)

In [94]:
sales

[{'sale_url': 'https://jewelry.ha.com/itm/estate-jewelry/earrings/aldo-cipullo-gold-earrings/a/172310-71028.s?ic4=ListView-Thumbnail-071515',
  'auction_number': '172310',
  'lot_number': '71028'},
 {'sale_url': 'https://jewelry.ha.com/itm/estate-jewelry/earrings/aldo-cipullo-gold-earrings/a/5160-64170.s?ic4=ListView-Thumbnail-071515',
  'auction_number': '5160',
  'lot_number': '64170'},
 {'sale_url': 'https://jewelry.ha.com/itm/estate-jewelry/bracelets/aldo-cipullo-for-cartier-gold-bracelet/a/172152-74002.s?ic4=ListView-Thumbnail-071515',
  'auction_number': '172152',
  'lot_number': '74002'},
 {'sale_url': 'https://jewelry.ha.com/itm/estate-jewelry/bracelets/sard-gold-bracelet-aldo-cipullo/a/5432-55026.s?ic4=ListView-Thumbnail-071515',
  'auction_number': '5432',
  'lot_number': '55026'},
 {'sale_url': 'https://jewelry.ha.com/itm/estate-jewelry/bracelets/gold-bracelet-aldo-cipullo-for-cartier/a/5248-55328.s?ic4=ListView-Thumbnail-071515',
  'auction_number': '5248',
  'lot_number': 

In [96]:
# BLOCKED BY CAPTCHA, change wait times, scroll sometimes, move cursor...
# Loop for each designer, navigate to each  page and get the product informations
for i in range(len(designers)):
    driver_.get(designers[i].get('page_url'))
    driver_.implicitly_wait(2)
    driver_.implicitly_wait(2)
    soup = bs(driver_.page_source, 'lxml') #Extract source page into beautiful soup
    
    # Call scroll_page_randomly with a random delay
    driver_ = scroll_page_randomly(driver_, np.random.randint(1, 6))
    
    # For each designer page 
    sales=[]
    for sale in soup.find_all(class_ = 'item-block list'):
        products={}
        #print(sale.contents)
        sale_url=sale.find('a')['href']

        #Extract Lot Number and Auction number
        sale_text= sale.find(class_ = 'item-info').text
        auction_number = re.search(r'Auction .*?(\d+)', sale_text).group(1)
        lot_number = re.search(r'Lot .*?(\d+)', sale_text).group(1)
        products["sale_url"] = sale_url
        products["auction_number"] = auction_number
        products["lot_number"] = lot_number

        sales.append(products)
        
    designers[i]["sales"] = sales



KeyboardInterrupt: 

In [98]:
designers

[{'name': 'A.Jaffe',
  'page_url': 'https://jewelry.HA.com/c/search.zx?header=Estate+Jewelry&search=archives&txtSearch=a.jaffe&ic22=JewelryMM-DesignerName-061120',
  'sales': [{'sale_url': 'https://jewelry.ha.com/itm/estate-jewelry/rings/sapphire-diamond-white-gold-ring-ajaffe/a/5316-55809.s?ic4=ListView-Thumbnail-071515',
    'auction_number': '5316',
    'lot_number': '55809'}]},
 {'name': 'Aaron Basha',
  'page_url': 'https://jewelry.HA.com/c/search.zx?header=Estate+Jewelry&search=archives&txtSearch=aaron+basha&ic22=JewelryMM-DesignerName-061120',
  'sales': [{'sale_url': 'https://jewelry.ha.com/itm/estate-jewelry/pendants-and-lockets/aaron-basha-diamond-platinum-pendant/a/172218-71029.s?ic4=ListView-Thumbnail-071515',
    'auction_number': '172218',
    'lot_number': '71029'},
   {'sale_url': 'https://jewelry.ha.com/itm/estate-jewelry/brooches-pins/aaron-basha-diamond-enamel-gold-brooch/a/5190-54031.s?ic4=ListView-Thumbnail-071515',
    'auction_number': '5190',
    'lot_number': '

# Part 3 : Extract Data
Links to pictures in full res

Description split in "item description", "condition report", "designer description"

Auction info : price, date, auction ID, bid source, lot ID, nb of bids, lot tracking activity, page views, 

Récupérer les url vers la page du produit, en fonction du auction ID, du lot ID, de l'url vers le produit

In [100]:
# OPTIONNAL : Get scroll height and slow scroll 
def scroll_page_randomly(driver):
    time.sleep(2)
    scroll_height = driver.execute_script("return document.body.scrollHeight")
    for i in range(0, scroll_height, np.random.randint(100, 500)):
        driver.execute_script(f"window.scrollTo(0, {i});")
        time.sleep(0.1)
    driver.execute_script(f"window.scrollTo(0, {scroll_height});")
    time.sleep(2)
    return driver


In [30]:
To know : 
    for images with url like https://dyn1.heritagestatic.com/lf?set=path%5B2%2F6%2F8%2F4%2F8%2F26848080%5D%2Csizedata%5B850x600%5D&call=url%5Bfile%3Aproduct.chain%5D
    full size can be accessed by removing "sizedata%5B850x600%5D" before "&call=url"

SyntaxError: invalid decimal literal (1831828876.py, line 2)

In [ ]:
# Part 4 : Create a database structure for image classification

In [ ]:
# Part 5 : Create a model. See what kaggle has to offer, and other platforms
Data Augmentation using available glossaries for 15k, 18k etc

Data Augmentation using image rotation etc etc

Train the model

Estimate a price? What could the inputs be other than image? Weight, size, color etc?

In [ ]:
# Part 6 : If it works, do the same for coins